# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

#print(len(full_data_rows_list))

#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Creating a Keyspace 
session.execute('''
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION =
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }''')

#### Set Keyspace

In [8]:
# Setting KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [18]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
#query = 'DROP TABLE IF EXISTS artist_song_length_session_338_item_4'
query = "CREATE TABLE IF NOT EXISTS artist_song_length_session_item"
query = query + """ 
(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"""
session.execute(query)
                    

In [19]:
# Inserting data into table for query 1

song_detail_file = 'event_datafile_new.csv'
with open(song_detail_file, encoding= 'utf8') as f:
    song_details = csv.reader(f)
    next(song_details)
    for row in song_details:
        query = '''
        INSERT INTO artist_song_length_session_item (sessionId, itemInSession, artist, song, length)
        '''
        query = query + """ VALUES (%s,%s,%s,%s,%s)"""
        
        session.execute(query, (int(row[8]), int(row[3]), row[0], row[9], float(row[5])) )

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## Adding in the SELECT statement to verify the data was entered into the table
query = """ SELECT artist, song, length FROM artist_song_length_session_338_item_4 WHERE sessionId = 338 and  itemInSession = 4"""

query_result = session.execute(query)

for row in query_result:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [13]:
#query = 'DROP TABLE IF EXISTS artist_song_length_session_182_userId_10'
#session.execute(query)

In [20]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS artist_song_length_session_userId"
query = query + """ 
(userId int, sessionId int, itemInSession int, artist text, song text, 
firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"""
session.execute(query)
                    

In [21]:
song_detail_file = 'event_datafile_new.csv'
with open(song_detail_file, encoding= 'utf8') as f:
    song_details = csv.reader(f)
    next(song_details)
    for row in song_details:
        query = '''
        INSERT INTO artist_song_length_session_userId 
        (userId, sessionId, artist, song, itemInSession, firstName, lastName)
        '''
        query = query + """ VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        
        session.execute(query, (int(row[10]), int(row[8]), row[0], row[9], int(row[3]), row[1], row[4]))

In [22]:
query = """ 
SELECT artist, song, firstname, lastname FROM artist_song_length_session_userId 
WHERE userId = 10 and  sessionId = 182
ORDER BY itemInSession DESC"""

query_result = session.execute(query)

for row in query_result:
    print(row.artist, row.song, row.firstname, row.lastname)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz


In [23]:
## Query 3: Printing every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """CREATE TABLE IF NOT EXISTS user_for_song_history """
query = query + "(song text, userId int,firstName text, lastName text, PRIMARY KEY (song, userId))"
session.execute(query)

                    

In [24]:
song_detail_file = 'event_datafile_new.csv'
with open(song_detail_file, encoding= 'utf8') as f:
    song_details = csv.reader(f)
    next(song_details)
    for row in song_details:
        query = '''
        INSERT INTO user_for_song_history (song, userId, firstName, lastName)
        '''
        query = query + """ VALUES (%s, %s, %s, %s)"""
        
        session.execute(query, (row[9], int(row[10]), row[1], row[4]))

In [25]:
# Perform query 3
query = "SELECT firstName, lastName FROM user_for_song_history WHERE song='All Hands Against His Own'"
query_result = session.execute(query)
for row in query_result:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [27]:
query = 'DROP TABLE IF EXISTS artist_song_length_session_userId'
session.execute(query)

#query = 'DROP TABLE IF EXISTS artist_song_length_session_182_userId_10'
#session.execute(query)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()